In [1]:
import torch.nn as nn 
from models import create_model 
import yaml 
from omegaconf import OmegaConf
from arguments import jupyter_parser
import torch 
import torch.nn.functional as F 

import logging
import wandb
import time
import os
import json

import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from collections import OrderedDict
from accelerate import Accelerator

from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, \
                            balanced_accuracy_score, classification_report, confusion_matrix, accuracy_score

from query_strategies import create_query_strategy, create_labeled_index
from models import ResNetSimCLR
from utils import NoIndent, MyEncoder
from omegaconf import OmegaConf

default_setting = './configs/benchmark/default_setting.yaml'
strategy_setting = './configs/benchmark/entropy_sampling.yaml'
cfg = jupyter_parser(default_setting, strategy_setting)


# # dataset 
from datasets import create_dataset
trainset, testset = create_dataset(
        dataset_name    = cfg.DATASET.dataset_name,
        datadir         = cfg.DATASET.datadir,
        img_size        = cfg.DATASET.img_size,
        mean            = cfg.DATASET.mean,
        std             = cfg.DATASET.std,
        aug_info        = cfg.DATASET.aug_info,
        anomaly_ratio   = cfg.DATASET.anomaly_ratio,
        **cfg.DATASET.get('params',{})
    )   

from torch.utils.data import DataLoader 
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=False)
# # model 
# from models.simclr import ResNetSimCLR
model = __import__('models').__dict__[cfg.MODEL.method](
    modelname = cfg.MODEL.modelname,
    out_dim   = cfg.MODEL.out_dim
).to('cuda')

# # criterion 
# criterion = __import__('criterions').__dict__[cfg.CRITERION.name](
#     batch_size = cfg.DATASET.batch_size,
#     **cfg.CRITERION.get('params',{})
#     )

# from accelerate import Accelerator
# accelerator = Accelerator() 
# exp_name           = 'temp'
# modelname          = cfg.MODEL.modelname
# pretrained         = cfg.MODEL.pretrained
# strategy_name           = cfg.AL.strategy
# n_start            = cfg.AL.n_start
# n_end              = cfg.AL.n_end
# n_query            = cfg.AL.n_query
# n_subset           = cfg.AL.n_subset
# init_method        = cfg.AL.init.method
# init_method_params = cfg.AL.init.get('params', {})
# trainset           = trainset
# validset           = testset
# testset            = testset
# criterion_name     = cfg.CRITERION.name
# criterion_params   = cfg.CRITERION.get('params',{})
# img_size           = cfg.DATASET.img_size
# num_classes        = cfg.DATASET.num_classes
# batch_size         = cfg.DATASET.batch_size
# test_batch_size    = cfg.DATASET.test_batch_size
# num_workers        = cfg.DATASET.num_workers
# opt_name           = cfg.OPTIMIZER.opt_name
# lr                 = cfg.OPTIMIZER.lr
# opt_params         = cfg.OPTIMIZER.get('params',{})
# epochs             = cfg.TRAIN.epochs
# log_interval       = cfg.TRAIN.log_interval
# use_wandb          = cfg.TRAIN.wandb.use
# savedir            = 'temp'
# seed               = cfg.DEFAULT.seed
# accelerator        = accelerator
# ckp_metric         = 'temp'
# cfg                = cfg

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [11]:
img, labels = next(iter(trainloader))
t_outputs, s_outputs = model(img.to('cuda'))
loss_map = np.zeros((len(trainloader.dataset), 64,64))

score_map = 1.
for t, s in zip(t_outputs, s_outputs):
    t,s = F.normalize(t,dim=1),F.normalize(s,dim=1) # channel wise normalize 
    sm = torch.sum((t - s) ** 2, 1, keepdim=True) # channel wise average 
    sm = F.interpolate(sm, size=(64, 64), mode='bilinear', align_corners=False) # Intepolation : (1,w,h) -> (1,64,64)
    score_map = score_map * sm 

In [17]:
def get_score_map(t_outputs:list, s_outputs:list) -> torch.Tensor:
    '''
    sm.shape = (B,1,64,64)
    '''
    score_map = 1.
    for t, s in zip(t_outputs, s_outputs):
        t,s = F.normalize(t,dim=1),F.normalize(s,dim=1) # channel wise normalize 
        sm = torch.sum((t - s) ** 2, 1, keepdim=True) # channel wise average 
        sm = F.interpolate(sm, size=(64, 64), mode='bilinear', align_corners=False) # Intepolation : (1,w,h) -> (1,64,64)
        score_map = score_map * sm 
    return score_map 

In [46]:
data = next(iter(testloader))

In [53]:
np.arange(130).sum()

8385

In [41]:
score_map.flatten(1).max(1)[0].detach().cpu().numpy()

tensor([5.3578, 5.2091, 5.2566, 4.8769, 5.0312, 5.6437, 5.2243, 5.1100, 5.6954,
        5.9759, 5.4641, 4.8042, 4.2613, 4.5210, 5.2040, 5.2469, 4.6537, 4.8127,
        5.5784, 5.6723, 4.7969, 4.9957, 4.0973, 5.4970, 4.7935, 5.0252, 4.4149,
        4.5798, 4.9115, 5.6856, 5.7163, 4.5442], device='cuda:0')

In [11]:
class FeatDist(nn.Module):
    def __init__(self, layers:list = [0,1,2,3]):
        super(FeatDist,self).__init__()
        self.layers = layers 
        
    def forward(self, t_outputs, s_outputs):
        t_outputs = [t_outputs[x] for x in self.layers]
        s_outputs = [s_outputs[x] for x in self.layers]
        
        total_loss = 0 
        for t,s in zip(t_outputs, s_outputs):
            t,s = F.normalize(t, dim=1), F.normalize(s, dim=1)
            total_loss += torch.sum((t.type(torch.float32) - s.type(torch.float32)) ** 2, 1).mean()
        return total_loss 

In [20]:
for t,s in zip(t_outputs, s_outputs):
    t,s = F.normalize(t, dim=1), F.normalize(s, dim=1)
    break
    # total_loss += torch.sum((t.type(torch.float32) - s.type(torch.float32)) ** 2, 1).mean()

In [24]:
t_outputs[1].shape

torch.Size([32, 128, 28, 28])

In [22]:
t.shape

torch.Size([32, 64, 56, 56])